In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

api_key = "ddfd01475e78ecc08703ba3677251cec"  

def scrape_with_scraperapi(url):
    scraper_url = f"http://api.scraperapi.com/?api_key={api_key}&url={url}"
    return requests.get(scraper_url)

years = list(range(2025, 2020, -1))
all_matches = []

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

for year in years:
    data = scrape_with_scraperapi(standings_url)
    soup = BeautifulSoup(data.text, "html.parser")
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = scrape_with_scraperapi(team_url)
        try:
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        except ValueError:
            continue

        soup = BeautifulSoup(data.text, "html.parser")
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        if not links:
            continue

        data = scrape_with_scraperapi(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
        except ValueError:
            continue

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        
        print(f"✅ Scraped: {team_name} ({year})")
        time.sleep(1)

match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv", index=False)


✅ Scraped: Liverpool (2025)
✅ Scraped: Arsenal (2025)
✅ Scraped: Manchester City (2025)
✅ Scraped: Chelsea (2025)
✅ Scraped: Newcastle United (2025)
✅ Scraped: Aston Villa (2025)
✅ Scraped: Nottingham Forest (2025)
✅ Scraped: Brighton and Hove Albion (2025)
✅ Scraped: Bournemouth (2025)
✅ Scraped: Brentford (2025)
✅ Scraped: Fulham (2025)
✅ Scraped: Crystal Palace (2025)
✅ Scraped: Everton (2025)
✅ Scraped: West Ham United (2025)
✅ Scraped: Manchester United (2025)
✅ Scraped: Wolverhampton Wanderers (2025)
✅ Scraped: Tottenham Hotspur (2025)
✅ Scraped: Leicester City (2025)
✅ Scraped: Ipswich Town (2025)
✅ Scraped: Southampton (2025)
✅ Scraped: Manchester City (2024)
✅ Scraped: Arsenal (2024)
✅ Scraped: Liverpool (2024)
✅ Scraped: Aston Villa (2024)
✅ Scraped: Tottenham Hotspur (2024)
✅ Scraped: Chelsea (2024)
✅ Scraped: Newcastle United (2024)
✅ Scraped: Manchester United (2024)
✅ Scraped: West Ham United (2024)
✅ Scraped: Crystal Palace (2024)
✅ Scraped: Brighton and Hove Albion (202

In [ ]:
match_df